In [3]:
#웹 크롤링이란?
#웹 페이지에서 가져와 필요한 부분만 추출하는 것
#html 태그 >> 시작 ~ 종료 태그
#시작 / 종료 사이에 들어가는 속성은 태그에 추가 정보를 제공
"""
<a href="https://n.news.naver.com/article/215/0001155913?ntype=RANKING" 
class="list_title nclicks('RBP.rnknws')">빗속에 떠난 푸바오…에버랜드 눈물바다</a>
"""
#케이스: <span class="list_time ">5시간전</span>

#블록: "<div class="list_content">

#아무 사이트나 크롤링 하면 안됨

                        

'\n<a href="https://n.news.naver.com/article/215/0001155913?ntype=RANKING" \nclass="list_title nclicks(\'RBP.rnknws\')">빗속에 떠난 푸바오…에버랜드 눈물바다</a>\n'

In [4]:
#리퀘스트
import requests
url = "https://n.news.naver.com/article/215/0001155913?ntype=RANKING"
agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
response = requests.get(url = url, headers = {"User-Agent":agent})

In [5]:
print(type(response))


<class 'requests.models.Response'>


In [6]:
print(response.status_code)

200


In [7]:
res_text = response.text
res_content = response.content

print(res_text[:200])
print('=' * 200)
print(res_content[:200])

<!doctype html>
<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36">
	<head>
		<meta charset="utf-8">
		
b'<!doctype html>\n<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36">\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t'


In [8]:
print(type(res_text)) # <class "str">
print(type(res_content)) # <class "bytes">

<class 'str'>
<class 'bytes'>


In [9]:
#파싱 / 뷰티풀 수프 이용
from bs4 import BeautifulSoup as BS
parsed_res_text = BS(res_text)
print(type(parsed_res_text))


<class 'bs4.BeautifulSoup'>


In [10]:
parsed_res_text = BS(res_text)
print(type(parsed_res_text))

<class 'bs4.BeautifulSoup'>


In [11]:
#find로 요소 찾기

# 첫 번째 a 태그
print(parsed_res_text.find("a"))

<a href="#ct">본문 바로가기</a>


In [14]:
#증권사 리포트
import requests
from bs4 import BeautifulSoup as BS

url = "http://www.paxnet.co.kr/stock/report/report?wlog_rpt=jm&menuCode=2222"
response = requests.get(url)
html = response.content
html = BS(html)

#li 태그를 li_list에 저장
div = html.find("div", attrs = {"class":"board-type"})
li_list = div.find_all("li")

In [6]:
#행 구분
check_str = '<strong class="color-cate"><span class="bracket">'
print(check_str in str(li_list[1]))
print(check_str in str(li_list[0]))


True
False


In [17]:
#종목
stock = li_list[1].find_all("a")[0].text
print(stock)

현대차


In [16]:
#제목
title = li_list[1].find_all("a")[1].text
print(title)

투싼의 전력! 하이브리드로 밀고 간다


In [18]:
#적정가격
price = li_list[1].find("div", attrs = {"class":"line3"}).text
price = price.replace('\r', '').replace('\t', '').replace('\n', '')
price = price.split(' ')[1].replace(',', '')[:-1]
display(price)

'320000'

In [19]:
#의견
opinion = li_list[1].find_all("div", attrs = {"class":"line3"})[1].text
opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
display(opinion)

'매수'

In [20]:
#증권사의 이름
trading_firm = li_list[1].find_all("div", attrs = {"class":"line3"})[2].text
display(trading_firm)

'하이투자증권'

In [11]:
#리포트 날짜
date = li_list[1].find_all("div", attrs = {"class":"line3"})[3].text
display(date)

'2024.03.27'

In [12]:
#날짜형식으로 변환
import pandas as pd
date = pd.to_datetime(date)
display(date)

Timestamp('2024-03-27 00:00:00')

In [8]:
#parsing 함수
def parsing_li(li):
    stock = li.find_all("a")[0].text # 종목명
    title = li.find_all("a")[1].text # 리포트 제목
    price = li.find("div", attrs = {"class":"line3"}).text # 적정가격
    price = price.replace('\r', '').replace('\t', '').replace('\n', '')
    price = price.split(' ')[1].replace(',', '')[:-1]
    opinion = li.find_all("div", attrs = {"class":"line3"})[1].text # 의견
    opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
    trading_firm = li.find_all("div", attrs = {"class":"line3"})[2].text # 증권사
    date = li.find_all("div", attrs = {"class":"line3"})[3].text # 날짜
    date = pd.to_datetime(date)
    
    return [stock, title, price, opinion, trading_firm, date]

In [24]:
#리포트 요약
print(parsing_li(li_list[1]))

['현대차', '투싼의 전력! 하이브리드로 밀고 간다', '320000', '매수', '하이투자증권', Timestamp('2024-03-27 00:00:00')]


In [25]:
#코드 확장

data = []
for li in li_list:
    if check_str in str(li):
        record = parsing_li(li)
        data.append(record)
data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])
display(data.head(10))

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,현대차,투싼의 전력! 하이브리드로 밀고 간다,320000,매수,하이투자증권,2024-03-27
1,엘앤에프,고객사 다변화로 실적 성장 가시성 확보,255000,매수,하이투자증권,2024-03-27
2,대원강업,구동 모터 코어 양산의 시작은 올해 말,-,-,하이투자증권,2024-03-27
3,코리아에프티,하이브리드 캐니스터는 단가가 2배,-,매수,하이투자증권,2024-03-27
4,우리기술,원전이 이끌고 방산 및 PSD가 민다,-,-,하이투자증권,2024-03-26
5,아모그린텍,4년은 ESS 25년은 ESS + 전기차,17000,매수,대신증권,2024-03-26
6,엔비티,긁지 않은 복권,-,-,SK증권,2024-03-26
7,나스미디어,지금입니다,-,-,SK증권,2024-03-26
8,세경하이테크,돋보일 필름 장인의 질주,13000,매수,메리츠증권,2024-03-26
9,엘앤에프,"SK On 향 계약, 전환점",-,-,메리츠증권,2024-03-26


In [ ]:
import time
import requests
from bs4 import BeautifulSoup

base_url = "http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue="
data = []

for page_no in range(1, 563):
    # 웹 페이지 소스 가져오기
    url = base_url.format(page_no)
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            time.sleep(1)
            break
        else:
            time.sleep(10 * 60)    
    html = response.content
    html = BeautifulSoup(html, 'html.parser')
    
    # 웹 페이지 소스에서 필요한 부분만 가져오기
    div = html.find("div", attrs={"class": "board-type"})
    li_list = div.find_all("li")
    for li in li_list:
        if check_str in str(li):
            record = parsing_li(li)
            data.append(record)

data = pd.DataFrame(data, columns=["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])
  

In [ ]:
#데이터 저장
data.to_csv("/Users/yang-eun-yeol/Desktop/my_quant/종목리포트.txt", sep = "\t", encoding = "euc-kr", index = False)

In [ ]:
#셀레니움
#크롬 드라이버
import selenium
from selenium import webdriver
driver = webdriver.Chrome() # 맥은 경로를 지정 안해도 되는 것 같다...
#네이버 뉴스로 이동
url = "https://news.naver.com/"
driver.get(url)
driver.implicitly_wait(10) 



In [ ]:
#페이지에 있는 요소 찾기
from selenium.webdriver.common.by import By
driver.get(url)
driver.implicitly_wait(10) 

In [ ]:
#잡 플래닛 기업 평점 수집 프로젝트

#기본세팅
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()

#회사 등록
corp = "SK하이닉스"
url = "https://www.jobplanet.co.kr/companies/20561/reviews/%EC%97%90%EC%8A%A4%EC%BC%80%EC%9D%B4%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4"

# url 들어가기
driver.get(url)
driver.implicitly_wait(10)  #로딩 10초 까지

# 팝업 창이 뜨면 끄기 
try:
    driver.find_element(By.CLASS_NAME, "btn_close_x_ty1").click() 
except:
    pass
review_list = driver.find_elements(By.CLASS_NAME, "content_wrap")
review = review_list[0]
date = review.find_elements(By.CLASS_NAME, "txt1")[-1].get_attribute('innerHTML')
print(date)
score_list = review.find_element(By.CLASS_NAME, "ctbody_lft") 
star_score = score_list.find_element(By.CLASS_NAME, "star_score").get_attribute('style')
print(star_score)

# 항목별 평점
bl_score_list = score_list.find_elements(By.CLASS_NAME, "bl_score")
welfare_salary = bl_score_list[0].get_attribute("style")
work_life_balance = bl_score_list[1].get_attribute("style")
culture = bl_score_list[2].get_attribute("style")
promotion_opp = bl_score_list[3].get_attribute("style")
management = bl_score_list[4].get_attribute("style")
print(welfare_salary, work_life_balance, culture, promotion_opp, management)

def convert_width_to_score(width):
    width_percent = int(width.split(' ')[1][:-2])
    score = int(width_percent / 20)
    return score

print(convert_width_to_score(star_score))

def extract_score_from_review(review):
    # 출력 초기화
    output = []
    # 리뷰에서 평점과 관련된 부분만 가져오기
    score_list = review.find_element(By.CLASS_NAME, "ctbody_lft")
    # 총 평점 가져와서 추가
    star_score = score_list.find_element(By.CLASS_NAME, "star_score").get_attribute('style')
    output.append(convert_width_to_score(star_score))
    # 항목별 평점 가져와서 추가
    other_score_elements = score_list.find_elements(By.CLASS_NAME, "bl_score")
    for element in other_score_elements:
        output.append(convert_width_to_score(element.get_attribute("style")))
    return output

print(extract_score_from_review(review))

#리뷰들크롤링

btn_pglast = driver.find_element(By.CLASS_NAME, "btn_pglast")
num_pages = int(btn_pglast.get_attribute("href").split('=')[1])

#다음 페이지
import time
def get_url(driver, url):
    while True:
        try:
            driver.get(url)
            driver.implicitly_wait(10) # 페이지가 전부 로딩될때까지 최대 10초 기다리기
            break
        except:
            driver.close()
            driver = webdriver.Chrome('../chromedriver.exe')
            time.sleep(60)

#리뷰를 모으는 코드

import pandas as pd
input_data = pd.read_excel("/Users/yang-eun-yeol/Desktop/my_quant/데이터/종목별_잡플래닛url.xlsx")
data = [] 

for corp, url in input_data[['종목명', 'URL']].values:
    # 초기 url 들어가기
    get_url(driver, url)
    # 전체 페이지 수 계산
    btn_pglast = driver.find_element(By.CLASS_NAME, "btn_pglast")
    num_pages = int(btn_pglast.get_attribute("href").split('=')[1])
    # 페이지 순회 시작
    for page in range(num_pages - 1):
        print("{}:{}/{}".format(corp, page+1, num_pages))
        current_url = url + "?page={}".format(page+1)
        get_url(driver, current_url)

        # 팝업 창이 뜨면 끄기 
        try:
            driver.find_element(By.CLASS_NAME, "btn_close_x_ty1").click()
            time.sleep(0.5)
        except:
            pass
        
        # 현재 페이지에서 데이터 가져오기
        while True:
            time.sleep(0.5)
            try:
                temp = []
                review_list = driver.find_elements(By.CLASS_NAME, "content_wrap")
                for review in review_list:
                    date = review.find_elements(By.CLASS_NAME, "txt1")[-1].get_attribute('innerHTML')
                    score = extract_score_from_review(review)
                    record = [corp, date] + score # score는 리스트임
                    temp.append(record)
                data += temp
                break
            except:
                driver.close()
                driver = webdriver.Chrome('../chromedriver.exe')
                get_url(driver, current_url)
                time.sleep(60)

data = pd.DataFrame(data, columns = ["기업", "날짜", "전체평점", "승진기회및가능성", "복지및급여", "업무와삶의균형", "사내문화", "경영진"])